In [ ]:
### CSV DATASET MAKING FROM OUR IMAGE DATASET ----


import os
import cv2
import mediapipe as mp
import pandas as pd

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True)
landmark_list = []

# Path to your dataset folders
base_path = r"C:\Users\USER\pushup_image_dataset"

for label in os.listdir(base_path):
    folder_path = os.path.join(base_path, label)
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = pose.process(img_rgb)

        if results.pose_landmarks:
            row = []
            for lm in results.pose_landmarks.landmark:
                row += [lm.x, lm.y, lm.z, lm.visibility]
            row.append(label)
            landmark_list.append(row)

pose.close()

# Create CSV from landmarks
columns = []
for i in range(33):
    columns += [f'x{i}', f'y{i}', f'z{i}', f'v{i}']
columns.append('label')

df = pd.DataFrame(landmark_list, columns=columns)
df.to_csv('pose_landmarks_dataset.csv', index=False)
print("CSV dataset created as 'pose_landmarks_dataset.csv'")


In [ ]:
### TRAINING THE CSV DATASET WITH RANDOM FOREST CLASSIFIER MODEL ------


import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib

# Load dataset
df = pd.read_csv("pose_landmarks_dataset.csv")
X = df.drop('label', axis=1)
y = df['label']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

# Save model
joblib.dump(model, 'pose_pushup_classifier.pkl')
print("Model saved as 'pose_pushup_classifier.pkl'")


In [5]:
### TESTING THE REAL LIFE PUSH UP AND COUNTING REPS --------


import cv2
import mediapipe as mp
import joblib
import numpy as np
import pyttsx3
import threading
import pygame

model = joblib.load("pose_pushup_classifier.pkl")

# Text-to-speech
engine = pyttsx3.init()
def speak(text):
    threading.Thread(target=lambda: engine.say(text) or engine.runAndWait()).start()

# # Background music
# pygame.mixer.init()
# pygame.mixer.music.load("think.mpeg")  # Replace with your own MP3 file
# pygame.mixer.music.play(-1)

mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(r"C:\Users\USER\pushup_videos\pp3.mp4")

prev_class = None
ready_to_count = False
reps = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = pose.process(frame_rgb)

    if result.pose_landmarks:
        landmarks = []
        for lm in result.pose_landmarks.landmark:
            landmarks += [lm.x, lm.y, lm.z, lm.visibility]

        prediction = model.predict([landmarks])[0]

        # Draw pose
        mp_drawing.draw_landmarks(frame, result.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Rep counting logic
        if prediction == "wrong":
            speak("Fix your posture")
            cv2.putText(frame, "Fix your posture!", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        elif prediction == "down" and prev_class == "up":
            ready_to_count = True
        elif prediction == "up" and prev_class == "down" and ready_to_count:
            reps += 1
            ready_to_count = False
            speak(f"Good rep. Total reps: {reps}")

        prev_class = prediction
        cv2.putText(frame, f"Pose: {prediction}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
        cv2.putText(frame, f"Reps: {reps}", (10, 130), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 3)

    cv2.imshow("Push-up Counter", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
pygame.mixer.music.stop()


C:\Users\USER\anaconda3\envs\ml_env\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\envs\ml_env\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\envs\ml_env\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\envs\ml_env\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\envs\ml_env\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomFore